In [2]:
!pip install python-dotenv


In [3]:
!pip install groq


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.3 MB/s eta 0:00:02
   ---------------- ----------------------- 0.8/2.0 MB 1.6 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 1.7 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 1.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 1.4 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 1.1 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 1.1 MB/s eta 0:00:01
   -------------------------------- --

In [4]:
from dotenv import load_dotenv
from groq import Groq
import json
import re


load_dotenv()

groq = Groq()

def classify_with_llm(log_msg):
    """
    Generate a variant of the input sentence. For example,
    If input sentence is "User session timed out unexpectedly, user ID: 9250.",
    variant would be "Session timed out for user 9251"
    """
    prompt = f'''Classify the log message into one of these categories: 
    (1) Workflow Error, (2) Deprecation Warning.
    If you can't figure out a category, use "Unclassified".
    Put the category inside <category> </category> tags. 
    Log message: {log_msg}'''

    chat_completion = groq.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        # model="llama-3.3-70b-versatile",
        model="deepseek-r1-distill-llama-70b",
        temperature=0.5
    )

    content = chat_completion.choices[0].message.content
    match = re.search(r'<category>(.*)<\/category>', content, flags=re.DOTALL)
    category = "Unclassified"
    if match:
        category = match.group(1)

    return category


if __name__ == "__main__":
    print(classify_with_llm(
        "Case escalation for ticket ID 7324 failed because the assigned support agent is no longer active."))
    print(classify_with_llm(
        "The 'ReportGenerator' module will be retired in version 4.0. Please migrate to the 'AdvancedAnalyticsSuite' by Dec 2025"))
    print(classify_with_llm("System reboot initiated by user 12345."))

Workflow Error
Deprecation Warning
Unclassified
